<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./images/DLI_Header.png"></a></div>

# Introduction to Content-Based Filtering

[Content-based filtering](http://recommender-systems.org/content-based-filtering/) is a simple and effective way to recommend items based on a single user's previous feedback. It works by comparing the categories of an item, such as genres for movies or properties of clothing.

## Objective
This notebook demonstrates:
* How to build a content-based filter
  * [1. A Sweet Example](#1.-A-Sweet-Example)
  * [2. Building a Content-Based Filter](#2.-Building-a-Content-Based-Filter)
* How to use a content-based filter for recommendation
  * [3. Making Predictions](#3.-Making-Predictions)
  * [4. Wrap Up](#4.-Wrap-Up)

## 1. A Sweet Example

Let's look at a toy example with candies. The example below is what's called a user-item interaction matrix, where rows represent users, columns represent items, and the cells represent how the respective user interacted with the respective item. In this case, the users have scored the candies they've tried on a 1 star to 5 star scale. Dashes, `-`, represent items they haven't had yet.

User | M&Ms | Skittles | Snickers | Laffy Taffy | Caramel Chew
-|-|-|-|-|-
Lara| 3 | - | 5 | - | -
Daisy| 4 | - | - | 1 | -
Alyx | - | - | - | - | 5
Sarah | - | 3 | 2 | - | -

Let's say we consult our subject matter experts to categorize and label these candies. They might give us another table that looks like this:

Candy | Chocolate | Round | Colorful | Fruity | Caramel | Chewy
- | - | - | - | - | - | -
M&Ms | 1 | 1 | 1 | 0 | 0 | 0
Skittles | 0 | 1 | 1 | 1 | 0 | 0
Snickers | 1 | 0 | 0 | 0 | 1 | 0
Laffy Taffy | 0 | 0 | 1 | 1 | 0 | 1
Caramel Chew | 0 | 0 | 0 | 0 | 1 | 1 |

There are many ways to build a content-based filter, but one quick and effective way is by finding the user's percent preference per category.

## 2. Building a Content-Based Filter

<img src="images/bayesian.png" width="500" height="500">

Let's start by loading our toy problem into a CUDA Data Frame using [cuDF](https://docs.rapids.ai/api/cudf/stable/), which is a drop-in replacement library for pandas that executes on the GPU as opposed to the CPU.

In [ ]:
import cudf

ratings = cudf.DataFrame({
    'user': ["Lara", "Lara", "Daisy", "Daisy", "Alyx", "Sarah", "Sarah"],
    'item': ["m_ms", "snickers", "m_ms", "laffy_taffy", "caramel_chew", "skittles", "snickers"],
    'rating': [3, 5, 4, 1, 5, 3, 2]})

ratings

In [ ]:
categories = cudf.DataFrame({
    'item': ["m_ms", "skittles", "snickers", "laffy_taffy", "caramel_chew"],
    'chocolate': [1, 0, 1, 0, 0],
    'round': [1, 1, 0, 0, 0],
    'colorful': [1, 1, 0, 1, 0],
    'fruity': [0, 1, 0, 1, 0],
    'caramel': [0, 0, 1, 0, 1],
    'chewy': [0, 0, 1, 1, 1]})

categories

We can join these DataFrames with [merge](https://docs.rapids.ai/api/cudf/stable/api_docs/api/cudf.DataFrame.merge.html) so we can find each user's preference for a category.

In [ ]:
joined_ratings = ratings.merge(categories, on="item", how="left")

joined_ratings

To make a Content-Based Filter, we'll start by multiplying each user's rating by the category for the item that rating belongs to.

In [ ]:
candy_classes = ['chocolate', 'round', 'colorful', 'fruity', 'caramel', 'chewy']

for candy_class in candy_classes:
    joined_ratings[candy_class] = (
        joined_ratings[candy_class] * joined_ratings["rating"])

joined_ratings

We can then use [groupby](https://docs.rapids.ai/api/cudf/stable/api_docs/api/cudf.DataFrame.groupby.html) to sum the columns for each user to find the total number of points given to each category. We can also drop the `rating` column as we won't need it anymore.

In [ ]:
user_profiles = joined_ratings.drop("rating", axis=1)
user_profiles = user_profiles.groupby(['user']).sum()

user_profiles

The goal of this filtering method is to find the user's percent preference for each category, so for that, we'll need the total amount of points given across every category for each user.

In [ ]:
user_profiles["total"] = user_profiles[candy_classes].sum(axis=1)

user_profiles

Finally, we can divide each category by the total to get a user's percentage preference for each class.

In [ ]:
for candy_class in candy_classes:
    user_profiles[candy_class] = (
        user_profiles[candy_class] / user_profiles["total"])
    
user_profiles = user_profiles.drop("total", axis=1)

user_profiles

## 3. Making Predictions

We can use these percentages to make new recommendations. For instance, Alyx has only had one candy: Caramel Chews. We can multiply her user profile across an item's categories to see what her predicted rating would be for that item.

With [CuPy](https://cupy.chainer.org/), we can efficiently do this for all users at once! CuPy is a GPU array library inspired by [NumPy](https://numpy.org/). Let's convert our DataFrames to CuPy arrays following the [conversion guide](https://docs.rapids.ai/api/cudf/stable/user_guide/10min-cudf-cupy.html).

In [ ]:
import cupy as cp

user_profiles_cupy = cp.fromDlpack(user_profiles.to_dlpack())

user_profiles_cupy

In [ ]:
categories_cupy = cp.fromDlpack(categories.drop("item", axis=1).to_dlpack())

categories_cupy

Let's make a prediction for one user first. We can take Alyx's profile and multiply it with the categories for each item.

In [ ]:
alyx_ratings = user_profiles_cupy[0] * categories_cupy

alyx_ratings

Then, we can [sum](https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.sum.html) each row to see the predicted preference for that corresponding row's item.

In [ ]:
alyx_ratings = cp.sum(alyx_ratings, axis=1)

alyx_ratings

Here, Alyx would get a high rating for Snickers and Caramel Chews, but low ratings for M&Ms and Skittles.

Now, let's do this for everyone. We can be efficient with our data by using [NumPy](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) broadcasting rules.

`user_profiles_cupy` and `categories_cupy` are 4x6 and 5x6 respectively. If we try to multiply them without doing anything, CuPy will give us a dimension mismatch error. We can use [cupy.expand_dims](https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.expand_dims.html) to make `user_profiles_cupy` 4x1x6. CuPy can then match the 1 to `categories_cupy`'s 5, and will multiply each user profile against each item's category listing, which is exactly what we need.

Since we have an extra dimension now, we'll do the sum against axis 2 instead of axis 1. We can verify that it worked because the first row will be the same as Alyx's prediction above.

In [ ]:
preferences = cp.expand_dims(user_profiles_cupy, axis=1) * categories_cupy
preferences = cp.sum(preferences, axis=2)

preferences

This is equivalent to taking the dot product.

In [ ]:
preferences = cp.dot(user_profiles_cupy, categories_cupy.transpose())
preferences

To get a predicted rating, we can scale these results by the same scale the users can use to rate items. If it's 1 - 5 stars, then we can multiply the range (5-1), and add the minimum rating (1).

In [ ]:
min_rating = 1
max_rating = 5
preferences = preferences * (max_rating - min_rating) + min_rating

preferences

It's time to bring it all together. Let's create a DataFrame to match each predicted rating to each user-item combo. Because we were careful to preserve order during the operations above, we can use [MultiIndex.from_product](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.from_product.html) in order to build a DataFrame for each user-item combination.

In [ ]:
user_ids = user_profiles.index.to_series().to_array()
category_ids = categories["item"].to_array()
predictions = cudf.MultiIndex.from_product([user_ids, category_ids], names=["user", "item"])
predictions = cudf.DataFrame(index = predictions).reset_index()

predictions

We can then use CuPy's [ravel](https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.ravel.html) function to flatten our predictions and add it as a column to our DataFrame.

In [ ]:
predictions["predicted_rating"] = cp.ravel(preferences)

predictions

Excellent! We now have predictions for each user-item interaction combo. We can sort the ratings for each user and recommend their top X items. There's one last important piece of the puzzle in that we don't want to recommend something the user has rated before. Let's add in the original ratings to do this.

Before moving on to the next step, let's compare the predicted rating with the actual rating. How did we do? Note where the algorithm went well and where it didn't and try to come up with an explanation for why.

In [ ]:
predictions = predictions.merge(ratings, how="left")

predictions

Finally, we can use [loc](https://docs.rapids.ai/api/cudf/stable/api_docs/api/cudf.DataFrame.loc.html) to locate the rows without a rating yet so we can serve recommendations to the user.

In [ ]:
predictions.loc[predictions['rating'].isnull()].drop("rating", axis=1)

## 4. Wrap Up

We didn't split the data into train and test because our example here is too small, so using a larger dataset is bound to introduce some interesting challenges. Now that we have a system that works with synthetic data, let's test our new knowledge on the real thing. 

Feeling ready? Please run the cell below to shut down the kernel before moving on to the <a href="1-04_content_based_real_data.ipynb">next notebook</a>.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./images/DLI_Header.png"></a></div>